In [3]:
# !pip install pyarrow

In [4]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

#import xgbtune 

In [5]:
df = pd.read_parquet("../data/dataset.parquet")
df

,date,open,high,low,close,adj close,volume,volitility_open_7,volitility_pct_change_7_open,high_low_7,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
0,2004-08-13,0.546429,0.558571,0.542857,0.550714,0.465064,328048000,NaN,NaN,0.015714,...,NaN,NaN,NaN,NaN,NaN,4,13,8,2004,33
1,2004-08-16,0.553571,0.566429,0.547143,0.549643,0.464160,435674400,NaN,NaN,0.019286,...,NaN,NaN,NaN,NaN,NaN,0,16,8,2004,34
2,2004-08-17,0.546071,0.555893,0.541964,0.551250,0.465517,323019200,NaN,NaN,0.013929,...,NaN,NaN,NaN,NaN,NaN,1,17,8,2004,34
3,2004-08-18,0.544821,0.568750,0.544464,0.566786,0.478637,364655200,NaN,NaN,0.024286,...,NaN,NaN,NaN,NaN,NaN,2,18,8,2004,34
4,2004-08-19,0.562679,0.568929,0.542143,0.548393,0.463104,388920000,NaN,NaN,0.026786,...,NaN,NaN,NaN,NaN,NaN,3,19,8,2004,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,2024-06-25,209.149994,211.380005,208.610001,209.070007,208.828293,56713900,3.384621,0.013439,2.770004,...,0.072649,0.007821,0.016588,0.017645,-0.008456,1,25,6,2024,26
5000,2024-06-26,211.500000,214.860001,210.639999,213.250000,213.003448,66213200,3.319522,0.014561,4.220001,...,0.028578,-0.007097,0.000053,0.006173,-0.006999,2,26,6,2024,26
5001,2024-06-27,214.690002,215.740005,212.350006,214.100006,213.852478,49772700,3.448615,0.015829,3.389999,...,0.005491,0.012392,0.001579,0.012218,0.004797,3,27,6,2024,26
5002,2024-06-28,215.770004,216.070007,210.300003,210.619995,210.376480,82542700,3.009810,0.013757,5.770004,...,-0.008168,-0.019148,0.005255,0.000632,-0.004892,4,28,6,2024,26


In [6]:
# target = "target_close"
target = "close_percent_change_1"

x = df.drop(columns=[target]).set_index("date")
y = df[["date", target]].set_index("date")
y

,close_percent_change_1
date,
2004-08-13,-0.001945
2004-08-16,0.002924
2004-08-17,0.028183
2004-08-18,-0.032451
2004-08-19,0.002930
...,...
2024-06-25,0.019993
2024-06-26,0.003986
2024-06-27,-0.016254


In [7]:
x

,open,high,low,close,adj close,volume,volitility_open_7,volitility_pct_change_7_open,high_low_7,high_close_7,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2004-08-13,0.546429,0.558571,0.542857,0.550714,0.465064,328048000,NaN,NaN,0.015714,NaN,...,NaN,NaN,NaN,NaN,NaN,4,13,8,2004,33
2004-08-16,0.553571,0.566429,0.547143,0.549643,0.464160,435674400,NaN,NaN,0.019286,0.015715,...,NaN,NaN,NaN,NaN,NaN,0,16,8,2004,34
2004-08-17,0.546071,0.555893,0.541964,0.551250,0.465517,323019200,NaN,NaN,0.013929,0.006250,...,NaN,NaN,NaN,NaN,NaN,1,17,8,2004,34
2004-08-18,0.544821,0.568750,0.544464,0.566786,0.478637,364655200,NaN,NaN,0.024286,0.017500,...,NaN,NaN,NaN,NaN,NaN,2,18,8,2004,34
2004-08-19,0.562679,0.568929,0.542143,0.548393,0.463104,388920000,NaN,NaN,0.026786,0.002143,...,NaN,NaN,NaN,NaN,NaN,3,19,8,2004,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-25,209.149994,211.380005,208.610001,209.070007,208.828293,56713900,3.384621,0.013439,2.770004,3.240005,...,0.072649,0.007821,0.016588,0.017645,-0.008456,1,25,6,2024,26
2024-06-26,211.500000,214.860001,210.639999,213.250000,213.003448,66213200,3.319522,0.014561,4.220001,5.789993,...,0.028578,-0.007097,0.000053,0.006173,-0.006999,2,26,6,2024,26
2024-06-27,214.690002,215.740005,212.350006,214.100006,213.852478,49772700,3.448615,0.015829,3.389999,2.490005,...,0.005491,0.012392,0.001579,0.012218,0.004797,3,27,6,2024,26


In [8]:
table_info = pd.DataFrame()

# Print columns that are not int, float, bool or category
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        table_info = pd.concat([table_info, pd.DataFrame({"Column Name": [col], "Data Type": [x[col].dtype]})])

table_info

,Column Name,Data Type
0,dividends_event_name,object
0,splits_event_name,object


In [9]:
# Convert columns to float if they are not int, float, bool or category. Handle Cannot cast DatetimeArray to dtype float64 (XGBoosted models cannot use strings, but categories as enumerated values)
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        try:
            x[col] = x[col].astype("float")
        except:
            # drop datetime columns
            x = x.drop(columns=[col])

            print(f"Dropped Column: {col}")

Dropped Column: dividends_event_name
Dropped Column: splits_event_name


In [10]:
# # Normalizing the features between 0 and 1
# y_scaler = MinMaxScaler()
# y = y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# x_scaler = MinMaxScaler()
# x = x_scaler.fit_transform(x)

# y

In [11]:
#    All Features   | Target
# +-----------------+---------+
# | x_train         | y_train | <- 85% of the data which is used for training
# |                 |         |
# +-----------------+---------+
# | x_test          | y_test  | <- 15% of the data which is used for testing
# +-----------------+---------+


# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, shuffle=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
# x_train

In [12]:
# y_test

In [13]:
x_train

,open,high,low,close,adj close,volume,volitility_open_7,volitility_pct_change_7_open,high_low_7,high_close_7,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2013-06-12,15.696429,15.758929,15.410714,15.435357,13.314726,265227200,0.220045,0.015643,0.348215,0.130359,...,0.014901,0.001790,-0.023926,-0.007882,-0.002699,2,12,6,2013,24
2020-05-14,76.127502,77.447502,75.382500,77.385002,75.487984,158929200,1.475216,0.019623,2.065002,0.535004,...,-0.016099,0.000707,0.007946,0.016687,-0.010259,3,14,5,2020,20
2006-07-10,1.989286,2.017500,1.946429,1.964286,1.658791,529345600,0.030197,0.013203,0.071071,0.038929,...,0.002720,0.004720,-0.025017,-0.012125,-0.003597,0,10,7,2006,28
2017-08-09,39.814999,40.317501,39.777500,40.264999,37.647499,104526000,0.885038,0.027421,0.540001,0.297501,...,-0.018897,-0.000466,0.001443,0.014611,-0.001477,2,9,8,2017,32
2005-03-08,1.496429,1.505714,1.432143,1.447500,1.222378,1021451200,0.042021,0.015655,0.073571,0.021072,...,0.034471,0.019455,0.001269,0.018231,0.018128,1,8,3,2005,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-11-17,7.360000,7.408571,7.321429,7.392857,6.243084,396513600,0.111644,0.011659,0.087142,0.028928,...,0.010914,0.020530,0.046877,0.001263,0.002011,1,17,11,2009,47
2020-10-12,120.059998,125.180000,119.279999,124.400002,121.569260,240226800,2.293206,0.025709,5.900002,8.209999,...,-0.007568,-0.041946,0.030000,0.033912,0.021074,0,12,10,2020,42
2022-07-05,137.770004,141.610001,136.929993,141.559998,139.814529,73353800,2.581878,0.019302,4.680008,2.680008,...,0.032761,0.006673,-0.038556,0.001747,0.001804,1,5,7,2022,27


In [14]:
import numpy

from itertools import count, takewhile
def frange(start, stop, step):
    return takewhile(lambda x: x< stop, count(start, step))

# list(frange(0, 1, 0.1))
numpy.linspace(5, 10, num=6).tolist()

[5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [15]:
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=150, max_depth=7, eta=0.05, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.05, subsample=1, colsampvle_bytree=.3)
model = xgb.XGBRegressor(n_estimators=500, max_depth=7, eta=0.05, subsample=0.9, colsample_bytree=0.4)
model.fit(x_train, y_train)

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV

tune_params = {
    # "n_estimators": [100, 150],
    # "max_depth": [5, 6,],
    # "learning_rate": [0.05, 0.1],

    "n_estimators": [400, 450, 500, 550, 600],
    "max_depth": [6, 7, 8, 9, 11],
    "learning_rate": [0.05, 0.1, 0.2, 0.3, 0.4,],
    "subsample": [0.7, 0.8, 0.9, 1.0, 1.1],
    "colsample_bytree": [0.1, 0.2, 0.3, 0.4, 0.5],

}

# {'subsample': 0.9, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.3}
# {'subsample': 0.8, 'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.2}

# xgb_model = xgb.XGBRegressor()
# # model = GridSearchCV(estimator=xgb_model, param_grid=tune_params)
# model = HalvingRandomSearchCV(estimator=xgb_model, param_distributions=tune_params)
# model.fit(x_train, y_train)

# print(model.best_params_)

pickle.dump(model, open("../model/xgboost_model.pkl", "wb")) # Save model as Python pickle object
model.save_model("../model/xgboost_model.json")

In [16]:
# print(model.best_params_)
# model


In [17]:
predicted = model.predict(x_test)
y_test["predicted"] = predicted
y_test

,close_percent_change_1,predicted
date,,
2017-07-13,0.008594,0.003425
2020-08-05,0.034889,0.010051
2023-01-06,0.004089,0.007213
2018-06-01,0.008358,0.003380
2008-02-22,0.002344,0.006831
...,...,...
2020-04-01,0.016687,0.022088
2023-01-27,-0.020078,0.006672
2023-03-14,0.002621,0.006839


In [18]:
# Get the root mean squared error (RMSE)
mse = np.mean((y_test["predicted"] - y_test[target]) ** 2)
rmse = np.sqrt(mse)
rmse

0.020639225211425125

In [19]:
# Get the mean absolute error (MAE)
mae = np.mean(np.abs(y_test["predicted"] - y_test[target]))
mae

0.013733777986632346

In [20]:
# 0.01455269590461311
# 0.013673832772506729
# 0.012685448862479572

In [21]:
# rmsle
rmsle = np.sqrt(np.mean(np.log1p(np.abs(y_test["predicted"] - y_test[target])) ** 2))
rmsle

0.020035980373427145

In [22]:
# Get the mean absolute percentage error (MAPE)
mape = np.mean(np.abs((y_test[target] - y_test["predicted"]) / y_test[target])) * 100
mape

inf

In [23]:
# Save predicted values
y_test.sort_index().to_parquet("../data/predicted.parquet")